In [ ]:
latest_gameweek = 5

In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path

import plotly.graph_objects as go
import matplotlib.pyplot as plt

pd.set_option('display.max_columns', 300)
#pd.set_option('display.max_rows', 100)

# Data

In [ ]:
filepath = Path(f'../data/predictions/gameweek{latest_gameweek}.csv')
projections = pd.read_csv(filepath, index_col=0)
display(projections.head())
display(projections.shape)

In [ ]:
filepath = Path('../data/fpl_df.csv')
fpl_df = pd.read_csv(filepath, index_col=0, low_memory=False)
fpl_df = fpl_df[fpl_df.season=='23-24']
display(fpl_df.head())
display(fpl_df.shape)

In [ ]:
df = fpl_df.groupby('name').last().reset_index()[['name', 'team_name', 'element_type', 'now_cost', 'points_per_game', 'total_points']]
df

# Analysis

In [ ]:
(projections[projections.gameweek==(latest_gameweek+1)]
 .groupby('name')
 .sum()
 .sort_values('expected_points', ascending=False).head(20))[['expected_points']]

In [ ]:
(projections[(projections.gameweek.isin([7,8])) & (projections.element_type==4)]
 .groupby('name')
 .sum()
 .sort_values('expected_points', ascending=False).head(20))[['expected_points']]

In [ ]:
projections[projections['name'].str.contains('Julián Álvarez')].head()

In [ ]:
my_gameweek = latest_gameweek+2

my_starting_11_names = ['Pickford', 'Estu', 'Chilwell', 'Dias', 'Saka', 'Fernandes', 'Rashford', 'Sterling',
                  'Maddison', 'Jackson', 'Haaland']
my_subs_names = ['Turner', 'Botman', 'Archer', 'i Bell']

my_starting_11 = pd.DataFrame()
for my_name in my_starting_11_names:
    my_starting_11 = pd.concat([
                    my_starting_11, 
                    projections[(projections.gameweek==my_gameweek) & (projections.name.str.contains(my_name))]
                    ])

my_subs = pd.DataFrame()
for my_name in my_subs_names:
    my_subs = pd.concat([
                    my_subs, 
                    projections[(projections.gameweek==my_gameweek) & (projections.name.str.contains(my_name))]
                    ])

expected_points = my_starting_11['expected_points'].sum() + my_starting_11['expected_points'].max()
display(f'Expected points: {expected_points}')

display(my_starting_11[['name', 'element_type', 'home', 'expected_points']])
display(my_starting_11.shape[0])

display(my_subs[['name', 'element_type', 'home', 'expected_points']])
display(my_subs.shape[0])

In [ ]:
(projections[(projections.gameweek==(latest_gameweek+1)) & (projections.element_type==1)]
 .groupby('name')
 .sum()
 .sort_values('expected_points', ascending=False).head(20))[['now_cost', 'expected_points']]

In [ ]:
players = ['Nicolas Jackson', 'Julián Álvarez']
colors = ['blue', 'white']
#players = ['Heung-Min Son', 'Bruno Borges Fernandes']
#colors = ['white', 'red']

fig = go.Figure()
for count, player in enumerate(players):

    my_projections = projections[projections['name']==player].sort_values(by='gameweek')

    fig.add_trace(
        go.Scatter(
            x=np.unique(my_projections['gameweek']),
            y=my_projections.groupby('gameweek').sum()['expected_points'],#.cumsum(),
            mode="markers+lines",
            marker=dict(color=colors[count]),
            name=player + ' - projected',
            #hovertext=('Opponent: ' + my_projections['opp_team'] + 
            #           ', was_home: ' + my_projections['was_home'].astype(str)),
            showlegend=True,
            ),
    )

fig.update_layout(
    #title="",
    template='plotly_dark',
    xaxis_title="gameweek",
    yaxis_title='projected_points',
    #showlegend=True
)

In [ ]:
for player in players:
    my_projections = projections[projections['name']==player].sort_values(by='gameweek')
    print(player)
    display(my_projections[['opponent_team','home','gameweek', 'expected_points']].set_index('gameweek'))